In [9]:
import pickle as pkl
import tqdm.notebook as tqdm
import cv2
import pandas as pd
import torch
import time

from numpy import random
import os
import tqdm.notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
import geopy.distance 
from IPython import display
from ipywidgets import Output

import custom_utils
torch.backends.cudnn.benchmark = True

In [10]:
device = 'cuda:0'
model = torch.load("trained_mlp/wsss_stereo.pt")
model.eval()

mlp_dual(
  (fc): Sequential(
    (0): Linear(in_features=8, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=1, bias=True)
    (7): ReLU()
  )
)

In [11]:
with open('processed_data\\1.pkl', 'rb') as f:
    detections, distances, calibration = pkl.load(f)
pred, gt = custom_utils.convert_to_model_format(detections, distances, calibration, scale_pred = model.scale_pred, scale_gt = model.scale_gt, skip_missing = True)

In [12]:
pred = pred[:-100]
gt = gt[:-100]

In [13]:
y = model(torch.from_numpy(pred).to(device).float()).cpu().detach().numpy() * 8

In [14]:
data = pd.DataFrame(pred, columns = ["Camera 1 Detection Box X-Centroid", "Camera 1 Detection Box Y-Centroid", "Camera 1 Detection Box Width", "Camera 1 Detection Box Height", "Camera 2 Detection Box X-Centroid", "Camera 2 Detection Box Y-Centroid", "Camera 2 Detection Box Width", "Camera 2 Detection Box Height"])
data['Ground Truth Distance'] = gt * 8
data['Prediction Stereo Camera'] = y.squeeze()

In [15]:
model = torch.load("trained_mlp/wsss_mono_view_1.pt")
model.eval()
y = model(torch.from_numpy(pred[:, :4]).to(device).float()).cpu().detach().numpy() * 8

In [16]:
data['Prediction Mono Camera 1'] = y.squeeze()

In [17]:
model = torch.load("trained_mlp/wsss_mono_view_2.pt")
model.eval()
y = model(torch.from_numpy(pred[:, 4:]).to(device).float()).cpu().detach().numpy() * 8

In [18]:
data['Prediction Mono Camera 2'] = y.squeeze()

In [19]:
y = data['Prediction Stereo Camera'].to_numpy()
gt = data["Ground Truth Distance"].to_numpy()
(100 * abs(y-gt)/gt).sum()/gt.shape[0]

0.13736580541562504

In [20]:
y = data['Prediction Mono Camera 1'].to_numpy()
gt = data["Ground Truth Distance"].to_numpy()
(100 * abs(y-gt)/gt).sum()/gt.shape[0]

0.2535459604418267

In [21]:
y = data['Prediction Mono Camera 2'].to_numpy()
gt = data["Ground Truth Distance"].to_numpy()
(100 * abs(y-gt)/gt).sum()/gt.shape[0]

0.2562244648969473

In [22]:
data.to_csv("data.csv")

In [10]:
import os

In [13]:
def callback():
    os.rename("tt.txt", "yy.txt")

In [14]:
import threading

timer = threading.Timer(5.0, callback)
timer.start()  # after 60 seconds, 'callback' will be called
